# Skills Checklist: Preparing for the Google TensorFlow Certification Exam

## TensorFlow developer skills

In [ ]:
# know how to program in python, resolve python issues, compile and run
# python programs in PyCharm


In [ ]:
# know how to find information about tensorflow APIs, including how
# to find guides and API references on tensorflow.org


In [ ]:
# know how to debug, investigate, and solve error messages from the
# tensorflow API


In [ ]:
# know how to search beyond tensorflow.org, as and when necessary, to
# solve your tensorflow questions


In [ ]:
# know how to create ML models using tensorflow where the model size
# is reasonable for the problem being solved


In [ ]:
# know how to save ML models and check the model file size

from pathlib import Path
Path('somefile.txt').stat().st_size


In [ ]:
# understand the compatibility discrepancies between different versions
# of tensorflow


In [ ]:
# ensure that inputs to a model are in the correct shape


In [ ]:
# ensure that you can match test data to the input shape of a 
# neural network


In [ ]:
# ensure that you can match output data of a neural network to
# specified input shape for test data


In [ ]:
# understand batch loading of data


In [ ]:
# use callbacks to trigger the end of training cycles
"""
see "Callback construction" below

In [ ]:
# use datasets from different sources


In [ ]:
# use datasets in different formats, including json and csv

# CSV FORMAT
insurance = pd.read_csv("https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/insurance.csv")
# turns it right into a pandas DataFrame!

# JSON FORMAT
import json

f = open("data.json",)
data = json.load(f) # returns JSON object as a dictionary


## Building and training neural network models using TensorFlow 2.x

In [ ]:
# use tensorflow 2.x
import tensorflow as tf
print(tf.__version__)

In [ ]:
# build, compile, and train ML models using tensorflow
from tensorflow.keras import layers

model = tf.keras.Sequential([
    layers.Dense(4,activation="relu"),
    layers.Dense(4,activation="relu"),
    layers.Dense(1,activation="sigmoid")
],name="model")

model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.fit(X_train,y_train,epochs=5)


### predicting results

In [ ]:
# use models to predict results
y_preds = model.predict(X_test)


In [ ]:
# build sequential models with multiple layers
"""
see above

In [ ]:
# build and train models for binary classification
"""
output layer must be layers.Dense(1,activation="sigmoid")
loss = "binary_crossentropy"

In [ ]:
# build and train models for multi-class categorization
"""
output layer must be layers.Dense(NUM_CLASSES,activation="softmax")
loss = "categorical_crossentropy"

In [ ]:
# plot loss and accuracy of a trained model
history = model.fit(...)
pd.DataFrame(history.history).plot(figsize=(10,7),xlabel="epochs")


## Image classification

steps:
1. **read in data**
2. **find ```train_dir``` and ```test_dir```**
3. **create ```train_datagen``` and ```test_datagen``` using ```ImageDataGenerator```, passing augmentation arguments for rescaling, etc.**
4. **create ```train_data``` and ```test_data``` using e.g. ```train_datagen.flow_from_directory()```**
    * ```shuffle=True```
    * ```batch_size=32```
    * ```class_model="binary"``` or ```"categorical"```
    * choose a target image size
5. **design model**
    * some combination of stacked ```layers.Conv2D()``` and ```layers.MaxPool2D()```
    * ```layers.Flatten()```
    * if binary, ```layers.Dense(1,activation="sigmoid")```
    * if multiclass, ```layers.Dense(num_classes,activation="softmax")```
6. **compile**
    * if binary, ```loss="binary_crossentropy"```
    * if multiclass, ```loss="categorical_crossentropy"```
    * ```metrics=["accuracy"]```
7. **fit model**
8. **reload stored best weights for model**
9. **evaluate**
10. **predict**


### getting image data (TFDS)

In [ ]:
import tensorflow_datasets as tfds

datasets_list = tfds.list_builders()
print("food101" in datasets_list)

# Load in the data (takes about 5-6 minutes in Google Colab)
(train_data, test_data), ds_info = tfds.load(name="food101", # target dataset to get from TFDS
                                             split=["train", "validation"], # what splits of data should we get? note: not all datasets have train, valid, test
                                             shuffle_files=True, # shuffle files on download?
                                             as_supervised=True, # download data in tuple format (sample, label), e.g. (image, label)
                                             with_info=True) # include dataset metadata? if so, tfds.load() returns tuple (data, ds_info)

ds_info.features
class_names = ds_info.features["label"].names

# Take one sample off the training data
train_one_sample = train_data.take(1)

# Output info about our training sample
for image, label in train_one_sample:
  print(f"Image shape: {image.shape}\nImage dtype: {image.dtype}")


# Make a function for preprocessing images
def preprocess_img(image, label):
    image = tf.image.resize(image, [224,224]) # reshape to img_shape
    return tf.cast(image, tf.float32), label # return (float32_image, label) tuple


In [ ]:
# batching datasets with TFDS
train_data = train_data.map(map_func=preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
train_data = train_data.shuffle(buffer_size=1000).batch(batch_size=32).prefetch(buffer_size=tf.data.AUTOTUNE)

test_data = test_data.map(preprocess_img, num_parallel_calls=tf.data.AUTOTUNE)
test_data = test_data.batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
# model with TFDS
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=(224,224,3))
# x = layers.Rescaling(1./255)(inputs) # only for ResNet
x = base_model(inputs) # training=False if rescaling layer
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(len(class_names))(x)
# separate activatiion of output layer so we can output float32 activations
outputs = layers.Activation("softmax",dtype=tf.float32)(x)
model = tf.keras.Model(inputs,outputs,name="model")

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.fit(train_data,
          epochs=3,
          steps_per_epoch=len(train_data),
          validation_data=test_data,
          validation_steps=len(test_data),
          callbacks=[])


### getting image data (tf.keras.datasets.fashion_mnist)

In [ ]:
# use datasets from tf.data.datasets
from tensorflow.keras.datasets import fashion_mnist
(train_data,train_labels),(test_data,test_labels) = fashion_mnist.load_data()

print(train_data.shape)
print(test_data.shape)

import matplotlib.pyplot as plt
plt.imshow(train_data[0],cmap=plt.cm.binary)
print(train_labels[0])

# you can look on the dataset's github page for label translation

### building and training a simple model (tf.keras.datasets.fashion_mnist)

In [ ]:
# build and train models to process real-world image datasets
# with fashion_mnist dataset:

model = tf.keras.Sequential([
    layers.Flatten(input_shape=(n,n)), # one image is n pixels x n pixels
    layers.Dense(4,activation="relu"),
    layers.Dense(4,activation="relu"),
    layers.Dense(NUM_CATEGORIES,activation="softmax")
],name="model")

# the provided labels are in integer format, not one-hot, so we use "sparse"
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.fit(x=train_data,
          y=train_labels,
          epochs=10,
          validation_data=(test_data,test_labels))

# normalized data works better
train_data = train_data/255. # if RGB values are not already 0 to 1
test_data = test_data/255.

# use LearningRateScheduler to plot loss across learning rates
# use plt.semilogx(lrs,history.history["loss"])


### getting image data (pizza_steak unzip)

In [ ]:
import zipfile

# Download zip file of pizza_steak images
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/pizza_steak.zip 

# Unzip the downloaded file
zip_ref = zipfile.ZipFile("pizza_steak.zip", "r")
zip_ref.extractall()
zip_ref.close()

num_steak_images_train = len(os.listdir("pizza_steak/train/steak"))


In [ ]:
# get the class names
import pathlib
import numpy as np

data_dir = pathlib.Path("pizza_steak/train/") # turn our training path into a Python path
class_names = np.array(sorted([item.name for item in data_dir.glob('*')])) # created a list of class_names from the subdirectories
print(class_names)


In [ ]:
# view image
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import random

random_image = random.sample(os.listdir("pizza_steak/train/steak/"),1)
img = mpimg.imread("pizza_steak/train/steak/"+random_image[0])
plt.imshow(img)


### using a real-world image file

In [ ]:
# use real-world images in different shapes and sizes (adjust in ImageDataGenerator as target_size=(n,n))
print(img.shape)

# input_shape definition for Sequential models
# input_shape = (batch_size,image_height,image_width,color_channels)

# import a single image and prepare it for prediction
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/images/03-steak.jpeg 
steak = mpimg.imread("03-steak.jpeg")
plt.imshow(steak)
plt.axis(False);

img = tf.io.read_file("03-steak.jpeg")
img = tf.image.decode_image(img,channels=3) # restrict to 3 color channels (because that's what our model trains on)
img = tf.image.resize(img,size=[224,224])
img = img/255.

# to compensate for the missing batch size in the input shape, use tf.expand_dims()
model.predict(tf.expand_dims(img,axis=0))


### using ImageDataGenerator (pizza_steak example)

In [ ]:
# build and train models to process real-world image datasets
# with ImageDataGenerator:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# binary problem:
train_dir = "pizza_steak/train/"
train_datagen = ImageDataGenerator(rescale=1./255)
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32,
                                               target_size=(224,224), # convert all images to my desired size
                                               class_mode="binary", # type of problem
                                               shuffle=True)

test_dir = "pizza_steak/test/"
val_datagen = ImageDataGenerator(rescale=1./255)
val_data = val_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224,224), # convert all images to my desired size
                                               class_mode="binary", # type of problem
                                               shuffle=True)

# can get a sample of the training data batch as:
images,labels = train_data.next()
len(images),len(labels)


In [ ]:
# for a multiclass problem, change class_mode to "categorical"
train_datagen.flow_from_directory(train_dir,
                                  batch_size=32,
                                  target_size=(224,224), # convert all images to my desired size
                                  class_mode="categorical", # type of problem
                                  shuffle=True)

In [ ]:
# understand how ImageDataGenerator labels images based on the 
# directory structure
"""
see above, "use ImageDataGenerator"

### an alternative to ImageDataGenerator (tf.keras.preprocessing.image_dataset_from_directory)

In [ ]:
# you can also use image_data_from_directory if you want to...
# it's faster, but they may not approve.

# multiclass example:
train_data = tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                 image_size=(224,224),
                                                                 label_model="categorical", # type of labels you have
                                                                 batch_size=32)

test_data = tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                                 image_size=(224,224),
                                                                 label_model="categorical", # type of labels you have
                                                                 shuffle=False,
                                                                 batch_size=32) # probably don't need to batch test data

train_data.class_names
images,labels = train_data.take(1)
# once again, labels are one-hot encoded.


### augmenting data

In [ ]:
# identify strategies to prevent overfitting, including augmentation
# and dropout


In [ ]:
# use image augmentation to prevent overfitting
# note: you don't augment test data

# to prevent overfitting, add complexity to ImageDataGenerator
train_datagen_augmented = ImageDataGenerator(rescale=1/255.,
                                             rotation_range=20,
                                             shear_range=0.2,
                                             zoom_range=0.2,
                                             width_shift_range=0.2,
                                             height_shift_range=0.2,
                                             horizontal_flip=True,
                                             vertical_flip=True)
                                            # etc.
                                            # detail: https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator


In [ ]:
# alternatively, if you aren't using ImageDataGenerator, you can build in a data augmentation layer
from tensorflow.keras.layers.experimental import preprocessing

data_augmentation = tf.keras.models.Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomRotation(0.2)
    # preprocessing.Rescaling(1./255) # needed for ResNet, not EfficientNet
],name="data_augmentation")

# may need tf.expand_dims(img,axis=0) to input a single image to data_augmentation

### defining convolutional neural networks with Conv2D and pooling layers

In [ ]:
# define convolutional neural networks with conv2D and pooling layers
# (binary)

model = tf.keras.models.Sequential([
    layers.Conv2D(filters=10,
                  kernel_size=3,
                  activation="relu",
                  input_shape=(224,224,3)), # see chosen target_size above
    layers.Conv2D(10,3,activation="relu"),
    layers.MaxPool2D(pool_size=2,padding="valid"), # padding could also be "same"
    layers.Conv2D(10,3,activation="relu"),
    layers.Conv2D(10,3,activation="relu"),
    layers.MaxPool2D(2),
    layers.Flatten(),
    # layers.Dense(100,activation="relu"), # could try this to improve performance...
    layers.Dense(1,activation="sigmoid")
],name="model")

# looks like ImageDataGenerator gives you one-hot labels...
# it also batches for you.
model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.fit(train_data,
          epochs=5,
          steps_per_epoch=len(train_data),
          validation_data=val_data,
          validation_steps=len(val_data),
          callbacks=[])

In [ ]:
# for a multiclass problem, just change model output layer
layers.Dense(NUM_CLASSES,activation="softmax")

# and change the compiling loss
loss="categorical_crossentropy"


In [ ]:
# a more simple conv2D and pooling model might be:
# (still binary)

model = tf.keras.models.Sequential([
    layers.Conv2D(10,3,activation="relu",input_shape=(224,224,3)),
    layers.MaxPool2D(pool_size=2), # reduces the number of features by half
    layers.Conv2D(10,3,activation="relu"),
    layers.MaxPool2D(),
    layers.Conv2D(10,3,activation="relu"),
    layers.MaxPool2D(),
    layers.Flatten(),
    layers.Dense(1,activation="sigmoid")
])

# with identical compiling and fitting protocols


In [ ]:
# understand how to use convolutions to improve your neural network
"""
convolution layers extract/learn the most important features from target images.
pooling layers reduce the dimensionality of learned image features.

### transfer learning

In [ ]:
# use pretrained models (transfer learning)

# accessing pretrained models:
# Resnet 50 V2 feature vector
resnet_url = "https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4"

# Original: EfficientNetB0 feature vector (version 1)
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"


In [ ]:
import tensorflow_hub as hub

feature_extractor_layer = hub.KerasLayer(model_url,
                                         input_shape=(224,224)+(3,),
                                         trainable=False, # to use pretrained weights
                                         name="feature_extractor_layer")

model = tf.keras.models.Sequential([
    feature_extractor_layer,
    layers.Dense(num_classes,activation="softmax")
],name="model")

# compile and fit as usual, with loss="categorical_crossentropy"


In [ ]:
# or, use the functional API
base_model = tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable = False

inputs = layers.Input(shape=(224,224,3))
# if using ResNet, need x = layers.experimental.preprocessing.Rescaling(1./255)(inputs)
x = base_model(inputs)
x = layers.GlobalAveragePooling2D()(x) # transforms a 4D tensor to a 2D tensor by averaging the values across the inner axes
outputs = layers.Dense(num_classes,activation="softmax")(x)
model = tf.keras.Model(inputs,outputs)

# if you add an augmentation layer between Input() and base_model(),
# you must use x = base_model(x,training=False)

# compile and fit as usual, with loss="categorical_crossentropy"


In [ ]:
# extract features from pre-trained models
for layer_number,layer in enumerate(base_model.layers):
  print(layer_number, layer.name)

# can access weights by indexing on layers
weights,biases = base_model.layers[1].get_weights()

# input tensor shape with the same number of dimensions as the output of EfficientNetB0
input_shape = (1,4,4,3)

# create a random tensor
input_tensor = tf.random.normal(input_shape)

# pass the random tensor through a global average pooling 2D layer
global_average_pooled_tensor = layers.GlobalAveragePooling2D()(input_tensor)

# this is the same as tf.reduce_mean(input_tensor,axis=[1,2]) (averaging across the middle axes)


### transfer learning fine-tuning

In [ ]:
# make sure you track history from the initial fit

for layer_number, layer in enumerate(base_model.layers):
  print(layer_number,layer.name,layer.trainable)

# to fine-tune,
base_model.trainable = True
NUM_LAYERS_UNFROZEN = 10

for layer in base_model.layers[:-NUM_LAYERS_UNFROZEN]:
  layer.trainable = False

# recompile
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(lr=0.0001), # reduce lr by factor of 10
              metrics=["accuracy"])

# refit model
initial_epochs = 5
fine_tune_epochs = initial_epochs + 5
model.fit(train_data,
          epochs=fine_tune_epochs,
          validation_data=test_data,
          initial_epoch=history.epoch[-1],
          validation_steps=len(test_data),
          callbacks=[])


## Natural language processing (NLP)

steps:
1. **read in data**
2. **turn train and test and/or val data into ```pd.DataFrame```(s)**
3. **shuffle dataframe contents**
4. **turn dataframe text into list(s)**
5. if binary:
    * turn dataframe labels ```.tolist()```

  if multiclass:
    * transform labels with ```OneHotEncoder```
    * transform labels with ```LabelEncoder```
    * find ```class_names``` and ```num_classes```

6. **create tokenization of text**
7. **create embedding for text**
8. **design model using these layers**

  if binary:
    * output ```activation="sigmoid"```, final shape is 1

  if multiclass:
    * output ```activation="softmax"```, final shape is ```num_classes```

9. **compile**

  if binary:
    * use ```binary_crossentropy``` and "accuracy" metric

  if multiclass:
    * use ```categorical_crossentropy``` (if using one-hot labels) and "accuracy" metric, or
    * use ```sparse_categorical_crossentropy``` (if using integer labels)

10. **fit model**

  if multiclass:
    * first use ```tf.data.Dataset.from_tensor_slices((text_list, labels_one_hot))``` and ```new_dataset.batch(BATCH_NUM).prefetch(tf.data.AUTOTUNE)``` to streamline fit

11. **reload stored best weights for model**
12. **evaluate**
13. **predict**

  if binary:
    * ```model_pred_probs = model.predict(test_sentences)``` and ```model_preds = tf.squeeze(tf.round(model_pred_probs))```

  if multiclass:
    * ```model_pred_probs = model.predict(test_dataset)``` and ```model_preds = tf.argmax(model_pred_probs,axis=1)```
    * in comparisons of preds and true labels, use ```test_labels_encoded```

In [ ]:
# build natural language processing systems using tensorflow


### getting NLP data (reading txt document lines)

In [ ]:
!git clone https://github.com/Franck-Dernoncourt/pubmed-rct
!ls pubmed-rct

# check what files are in the 20k dataset
!ls pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/

# start our experiments using the 20k dataset with numbers replaced by @
data_dir = "/content/pubmed-rct/PubMed_20k_RCT_numbers_replaced_with_at_sign/"

# check all of the filenames in the target directory
import os
filenames = [data_dir+filename for filename in os.listdir(data_dir)]
filenames

In [ ]:
# def get_lines(filename):
#   with open(filename, "r") as f:
#       return f.readlines()

f = open(filename,"r")
lines = f.readlines()

# create a dictionary containing details like "line_number", "target", "text"

# # you can use line parsing tools like:
# for line in input_lines:
#   if line.startswith("###"):
#   elif line.isspace():

# abstract_line_split = abstract_lines.splitlines() # split text 

# for abstract_line_number,abstract_line in enumerate(abstract_line_split)
# target_text_split = abstract_line.split("\t")
# target_text_split[0].lower()

# # this is in 09. milestone project 
# train_samples = preprocess_text_with_line_numbers(data_dir + "train.txt")
# val_samples = preprocess_text_with_line_numbers(data_dir + "dev.txt") # dev is another name for validation set
# test_samples = preprocess_text_with_line_numbers(data_dir + "test.txt")


In [ ]:
# create a dictionaries train_samples, val_samples, and test_samples
# then, make them into pandas dataframes:
train_df = pd.DataFrame(train_samples)
val_df = pd.DataFrame(val_samples)
test_df = pd.DataFrame(test_samples)

train_sentences = train_df["text"].tolist()
val_sentences = val_df["text"].tolist()
test_sentences = test_df["text"].tolist()


### making numeric labels: one-hot encoding and label encoding of labels (multiclass categorization)

In [ ]:
# extract labels and one-hot encode them
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder(sparse=False)
train_labels_one_hot = one_hot_encoder.fit_transform(train_df["target"].to_numpy().reshape(-1,1))
val_labels_one_hot = one_hot_encoder.fit_transform(val_df["target"].to_numpy().reshape(-1,1))
test_labels_one_hot = one_hot_encoder.fit_transform(test_df["target"].to_numpy().reshape(-1,1))


In [ ]:
# extract labels and integer encode them
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_df["target"].to_numpy())
val_labels_encoded = label_encoder.fit_transform(val_df["target"].to_numpy())
test_labels_encoded = label_encoder.fit_transform(test_df["target"].to_numpy())


In [ ]:
# you can get class names if you transform with LabelEncoder!!!
class_names = label_encoder.classes_
num_classes = len(class_names)


### getting NLP data (download and unzip csv)

In [ ]:
# prepare text to use in tensorflow models
# Download data (same as from Kaggle)
!wget "https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip"

import zipfile

# Unzip data
zip_ref = zipfile.ZipFile("nlp_getting_started.zip", "r")
zip_ref.extractall()
zip_ref.close()

# Turn .csv files into pandas DataFrames
import pandas as pd
train_df = pd.read_csv("train.csv")
test_df = pd.read_csv("test.csv")
train_df.head()


### shuffling a training dataframe

In [ ]:
# Shuffle training dataframe
train_df_shuffled = train_df.sample(frac=1, random_state=42) # shuffle with random_state=42 for reproducibility


### creating NLP train/val sets with train_test_split

In [ ]:
from sklearn.model_selection import train_test_split

# Use train_test_split to split training data into training and validation sets
train_sentences, val_sentences, train_labels, val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size=0.1, # dedicate 10% of samples to validation set
                                                                            random_state=42) # random state for reproducibility
                                                                            

### TensorFlow Hub USE encoding

In [ ]:
import tensorflow_hub as hub

sentence_encoder_layer = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4",
                                        input_shape=[], # shape of inputs coming to our model 
                                        dtype=tf.string, # data type of inputs coming to the USE layer
                                        trainable=False, # keep the pretrained weights (we'll create a feature extractor)
                                        name="USE")

# you can instead fine-tune the TF Hub USE by setting trainable=True

# to be used specially in a model:
model = tf.keras.models.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation="relu"),
    layers.Dense(1,activation="sigmoid")
],name="model")

# compile and fit as below ("building a binary model")


### tokenization options

In [ ]:
# use word embeddings in your tensorflow model


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
# Note: in TensorFlow 2.6+, you no longer need "layers.experimental.preprocessing"
# you can use: "tf.keras.layers.TextVectorization", see https://github.com/tensorflow/tensorflow/releases/tag/v2.6.0 for more

# for max_tokens, either multiples of 10000 or exact number of unique words in your text
max_vocab_length = 10000 # max number of words to have in our vocabulary

# average number of tokens per unit
# something like: round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))
max_length = 15 # max length our sequences will be (e.g. how many words from a Tweet does our model see?)


In [ ]:
# tokenization layer: binary example
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_mode="int",
                                    output_sequence_length=max_length)

text_vectorizer.adapt(train_sentences)


In [ ]:
# tokenization layer: multiclass example (token)
text_vectorizer = TextVectorization(max_tokens=max_vocab_length,
                                    output_sequence_length=max_length)

text_vectorizer.adapt(train_sentences)

# max_length for sentences may be found as e.g. int(np.percentile(sent_lens,95))

In [ ]:
# tokenization layer: multiclass example (char)

def split_chars(text):
  return " ".join(list(text))

train_chars = [split_chars(sentence) for sentence in train_sentences]
val_chars = [split_chars(sentence) for sentence in val_sentences]
test_chars = [split_chars(sentence) for sentence in test_sentences]

import string
alphabet = string.ascii_lowercase + string.digits + string.punctuation

num_char_tokens = len(alphabet)+2 (to include space and OOV)
# max_length for chars may be found as e.g. int(np.percentile(char_lens,95))

char_vectorizer = TextVectorization(max_tokens=num_char_tokens,
                                    output_sequence_length=max_length,
                                    standardize="lower_and_strip_punctuation")

char_vectorizer.adapt(train_chars)


### embedding options

In [ ]:
from tensorflow.keras import layers

# embedding layer: binary example
embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             embeddings_initializer="uniform", # default, intialize randomly
                             input_length=max_length, # how long is each input
                             name="embedding") 


In [ ]:
# embedding layer: multiclass example (token)
embedding = layers.Embedding(input_dim=max_vocab_length, # set input shape
                             output_dim=128, # set size of embedding vector
                             # use masking to handle variable sequence length
                             mask_zero=True,
                             name="embedding")


In [ ]:
# embedding layer: multiclass example (char)
char_embed = layers.Embedding(input_dim=num_char_tokens,
                              output_dim=25,
                              mask_zero=False, # DON'T use masks
                              name="embedding")


### hybrid embedding (multiclass categorization)

e.g. take token and character-level sequences as input and produce sequence label probabilities as output

In [ ]:
# set up token inputs/model (e.g. USE tokenization/embedding)
token_inputs = layers.Input(shape=[],dtype="string")
x = tf_hub_embedding_layer(token_inputs)
token_outputs = layers.Dense(128,activation="relu")(x)
token_model = tf.keras.Model(token_inputs,token_outputs,name="token_model")

# set up char inputs/model (e.g. Bidirectional LSTM)
char_inputs = layers.Input(shape=(1,),dtype="string")
x = char_vectorizer(char_inputs)
x = char_embed(x)
char_outputs = layers.Bidirectional(layers.LSTM(25))(x)
char_model = tf.keras.Model(char_inputs,char_outputs,name="char_model")

# concatenate token and char inputs
token_char_concat = layers.Concatenate()([token_model.output,
                                          char_model.output])

# create output layers (including Dropout)
combined_dropout = layers.Dropout(0.5)(token_char_concat)
combined_dense = layers.Dense(200,activation="relu")(combined_dropout)
final_dropout = layers.Dropout(0.5)(combined_dense)
output_layer = layers.Dense(num_classes)

# construct model with char and token inputs
model = tf.keras.Model(inputs=[token_model.input,
                               char_model.input],
                       outputs=output_layer,
                       name="model")

# model.summary()
# from tensorflow.keras.utils import plot_model
# plot_model(model)

# compile as normal
# could use loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.2)


In [ ]:
train_char_token_data = tf.data.Dataset.from_tensor_slices((train_sentences,train_chars))
train_char_token_labels = tf.data.Dataset.from_tensor_slices(train_labels_one_hot)
train_char_token_dataset = tf.data.Dataset.zip((train_char_token_data,train_char_token_labels))
train_char_token_dataset = train_char_token_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

# do the same for val_char_token_dataset and test_char_token_dataset

model.fit(train_char_token_dataset,
          epochs=3,
          steps_per_epoch=len(train_char_token_dataset),
          validation_data=val_char_token_dataset,
          validation_steps=len(val_char_token_dataset),
          callbacks=[])


### building a binary model

In [ ]:
# build models that identify the category of a piece of text using
# binary categorization


In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string") # inputs are 1D strings
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x) # binary output
model = tf.keras.Model(inputs,outputs,name="model")

model.compile(loss="binary_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.fit(x=train_sentences,
          y=train_labels,
          epochs=5,
          validation_data=(val_sentences,val_labels),
          callbacks=[])

# can check embedding weights: embedding.weights
# embed_weights = model.get_layer("embedding").get_weights()[0]

# you get a binary prediction from rounding the output number
model_pred_probs = model.predict(val_sentences)
model_preds = tf.squeeze(tf.round(model_pred_probs))


### building a multiclass model

In [ ]:
# build models that identify the category of a piece of text using
# multiclass categorization

train_dataset = tf.data.Dataset.from_tensor_slices((train_sentences,train_labels_one_hot))
train_dataset = train_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

val_dataset = tf.data.Dataset.from_tensor_slices((val_sentences,val_labels_one_hot))
val_dataset = val_dataset.batch(32).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_sentences,test_labels_one_hot))
test_dataset = test_dataset.batch(32).prefetch(tf.data.AUTOTUNE)


In [ ]:
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embeddings(x)
x = layers.Conv1D(64,kernel_size=5,padding="same",activation="relu")(x)
x = layers.GlobalAveragePooling1D()(x)
outputs = layers.Dense(num_classes,activiation="softmax")(x)
model = tf.keras.Model(inputs,outputs,name="model")

model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

model.fit(train_dataset,
          epochs=3,
          steps_per_epoch=len(train_dataset),
          validation_data=val_dataset,
          validation_steps=len(val_dataset),
          callbacks=[])

# you get a categorical prediction from argmax
model_pred_probs = model.predict(test_dataset)
model_preds = tf.argmax(model_pred_probs,axis=1)

# see char equivalent in 


### using an LSTM in an NLP model

can also make bidirectional by nesting layers:
```
x = layers.Bidirectional(layers.LSTM(64))(x)
```

In [ ]:
# use LSTMs in your model to classify text for either binary or 
# multiclass categorization

# binary LSTM:
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# if stacking RNN/LSTM layers, 
# x = layers.LSTM(64,return_sequences=True)(x) # to return vector for each word
x = layers.LSTM(64)(x) # return vector for whole sequence
# could add: x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model = tf.keras.Model(inputs,outputs,name="model")

# compile and fit as above ("building a binary model")


### using a GRU in an NLP model

can also make bidirectional by nesting layers:
```
x = layers.Bidirectional(layers.GRU(64))(x)
```

In [ ]:
# add RNN and GRU layers to your model

# binary GRU
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
# if stacking RNN/GRU layers, 
# x = layers.GRU(64,return_sequences=True)(x) # to return vector for each word
x = layers.GRU(64)(x) # return vector for whole sequence
# could add: x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model = tf.keras.Model(inputs,outputs,name="model")

# compile and fit as above ("building a binary model")


### using a CNN in an NLP model

In [ ]:
# use RNNs, LSTMs, GRUs, and CNNs in models that work with text
inputs = layers.Input(shape=(1,),dtype="string")
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters=32,kernel_size=5,activation="relu")(x)
x = layers.GlobalMaxPool1D()(x)
# could add: x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model = tf.keras.Model(inputs,outputs,name="model")

# compile and fit as above ("building a binary model")

# for a multiclass model, see above ("building a multiclass model")


### text generation

https://www.tensorflow.org/text/tutorials/text_generation

In [ ]:
# train LSTMs on existing text to generate text (such as songs and poetry)

# see other colab notebook: text_generation


## Time series, sequences, and predictions

steps:
1. **read in data**
2. **turn data into ```pd.DataFrame```**
3. **DO NOT shuffle dataframe contents**
4. **turn dataframe content into lists**
    * create ```X``` as price windows list
    * create ```y``` as corresponding price list
    * if multivariate, make appropriate n other lists

  list creation approach options include:
    * windowed array approach
    * copy of ```pd.Dataframe``` and addition of columns defining shifting windows

5. **split into train and test sets with a percentage-based split size**
    * earlier datapoints must constitute the train dataset
    * later datapoints must constitute the test dataset

6. **design model**
    * output layer: ```layers.Dense(HORIZON,activation="linear")```

7. **compile**
    * use ```mae``` for loss

8. **fit model**
    * first use ```tf.data.Dataset.from_tensor_slices((windows, prices))``` and ```new_dataset.batch(BATCH_NUM).prefetch(tf.data.AUTOTUNE)``` to create fast-loading batched datasets

9. **reload stored best weights for model**
10. **evaluate**
11. **predict**
    * ```model_preds = model.predict(test_windows)```
    * for predictions with horizons > 1, ```model_preds = tf.squeeze(model_preds)```

12. **forecast**
    * use iterations of model training, prediction, and dataset update for however many future steps you'd like
    * you might try first smoothing data with a trailing window of some width (perhaps beginning with 3?)

In [ ]:
# train, tune, and use time series, sequence, and prediction models


### getting time series data with pd

In [ ]:
# prepare data for time series learning


In [ ]:
# preprocess data to get it ready for use in a model

# data from GitHub:
# Note: you'll need to select "Raw" to download the data in the correct format
!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/BTC_USD_2013-10-01_2021-05-18-CoinDesk.csv 

# time series with pandas:
import pandas as pd
df = pd.read_csv("/content/BTC_USD_2013-10-01_2021-05-18-CoinDesk.csv", 
                 parse_dates=["Date"], 
                 index_col=["Date"]) # parse the date column (tell pandas column 1 is a datetime)

bitcoin_prices = pd.DataFrame(df["Closing Price (USD)"]).rename(columns={"Closing Price (USD)": "Price"})


In [ ]:
timesteps = bitcoin_prices.index.to_numpy()
prices = bitcoin_prices["Price"].to_numpy()


### time series train/test sets with price list (univariate)


In [ ]:
# prepare features and labels


In [ ]:
WINDOW_SIZE = 7
HORIZON = 1

# create a window of specific window_size
window_step = np.expand_dims(np.arange(WINDOW_SIZE+HORIZON),axis=0)

# create a 2D array of multiple window steps
window_indices = window_step + np.expand_dims(np.arange(len(prices)-(WINDOW_SIZE+HORIZON-1)),axis=0).T

# index on the time series with 2D array
windowed_array = prices[window_indices]

# finally, label windowed data
full_windows, full_labels = windowed_array[:,:-HORIZON], prices[:,-HORIZON:]

# need train data to be early data and test data to be later data
split_size = int(0.8*len(prices)) # 80% train, 20% test

train_windows, train_labels = full_windows[:split_size], full_labels[:split_size]
test_windows, test_labels = full_windows[split_size:], full_labels[split_size:]


### fast-loading batched datasets with price list (univariate)

In [ ]:
BATCH_NUM = 128

train_windows_data = tf.data.Dataset.from_tensor_slices(train_windows)
train_labels_data = tf.data.Dataset.from_tensor_slices(train_labels)
train_dataset = tf.data.Dataset.zip((train_windows_data,train_labels_data))
train_dataset = train_dataset.batch(BATCH_NUM).prefetch(tf.data.AUTOTUNE)

test_windows_data = tf.data.Dataset.from_tensor_slices(test_windows)
test_labels_data = tf.data.Dataset.from_tensor_slices(test_labels)
test_dataset = tf.data.Dataset.zip((test_windows_data,test_labels_data))
test_dataset = test_dataset.batch(BATCH_NUM).prefetch(tf.data.AUTOTUNE)


### time series train/test sets with pd (univariate _or_ multivariate)

In [ ]:
bitcoin_prices_windowed = bitcoin_prices.copy()

# for each "Price" and "Block_Reward" in bitcoin_prices, we need WINDOW_SIZE columns of values
for i in range(WINDOW_SIZE):
  bitcoin_prices_windowed[f"Price+{i+1}"] = bitcoin_prices_windowed["Price"].shift(periods=i+1)

split_size = int(len(y)*0.8)

# need the labels...
y = bitcoin_prices_windowed.dropna()["Price"].astype(np.float32)

###

# to consider both variables as joint inputs to the same layer:
X = bitcoin_prices_windowed.dropna().drop("Price",axis=1).astype(np.float32) # (same as usual, just happens to have an extra column)
X_train, y_train = X[:split_size], y[:split_size]
X_test, y_test = X[split_size:], y[split_size:]

# to consider variables as inputs to separate layers:
X = bitcoin_prices_windowed.dropna().drop("Price",axis=1).astype(np.float32)
X_price_window = X.dropna().drop("Block_Reward",axis=1).astype(np.float32)
X_block_reward = X.dropna()["Block_Reward"].astype(np.float32)

X_price_window_train, X_block_reward_train, y_train = X_price_window[:split_size], X_block_reward[:split_size], y[:split_size]
X_price_window_test, X_block_reward_test, y_test = X_price_window[split_size:], X_block_reward[split_size:], y[split_size:]

###


### fast-loading batched datasets with pd (univariate _or_ multivariate)

In [ ]:
BATCH_NUM = 128

y_train_data = tf.data.Dataset.from_tensor_slices(y_train)
y_test_data = tf.data.Dataset.from_tensor_slices(y_test)

###

# to consider both variables as joint inputs to the same layer:
X_train_data = tf.data.Dataset.from_tensor_slices(X_train)
X_test_data = tf.data.Dataset.from_tensor_slices(X_test)

# to consider variables as inputs to separate layers:
X_train_data = tf.data.Dataset.from_tensor_slices((X_price_window_train,X_block_reward_train))
X_test_data = tf.data.Dataset.from_tensor_slices((X_price_window_test,X_block_reward_test))

###

train_dataset = tf.data.Dataset.zip((X_train_data,y_train_data))
train_dataset = train_dataset.batch(BATCH_NUM).prefetch(tf.data.AUTOTUNE)

test_dataset = tf.data.Dataset.zip((X_test_data,y_test_data))
test_dataset = test_dataset.batch(BATCH_NUM).prefetch(tf.data.AUTOTUNE)


### multivariate time series models

In [ ]:
model = 

# to consider both variables as joint inputs to the same layer:

# to consider variables as inputs to separate layers:

price_window_model

block_reward_model


# ordinary compiling and fitting using train_dataset (and test_dataset)


### time series prediction

In [ ]:
# train models to predict values for both univariate and multivariate
# time series


In [ ]:
# understand Mean Absolute Error (MAE) and how it can be used to
# evaluate accuracy of sequence models

model = tf.keras.Sequential([
    layers.Dense(128,activation="relu"),
    layers.Dense(HORIZON,activation="linear") # activation="linear" is unnecessary
],name="model")

model.compile(loss="mae",
              optimizer=tf.keras.optimizers.Adam())

model.fit(x=train_windows,
          y=train_labels,
          epochs=100,
          batch_size=128,
          validation_data=(test_windows,test_labels),
          callbacks=[create_model_checkpoint("model")]) # to save best model, see callback section below
# -or-

model.fit(train_dataset,
          epochs=100,
          steps_per_epoch=len(train_dataset),
          validation_data=test_dataset,
          validation_steps=len(test_dataset)
          callbacks=[create_model_checkpoint("model")]) # to save best model, see callback section below

# reload best model
model = tf.keras.models.load_model("model_experiments/model")

# assess model
model.evaluate(test_windows,test_labels) # or model.evaluate(test_dataset)

# predict with model
model_preds = model.predict(test_windows) # or model.predict(test_dataset)

# predictions with bigger horizons
model_preds = tf.squeeze(model_preds)

# if you have shape problems, try tf.squeeze(test_labels) when comparing
# test_labels and model_preds

# see "what is MAE" below for a way to evaluate bigger window bois


What is MAE?

mean absolute error: forecast methods minimizing the MAE lead to forecasts of the median.

```
mae = tf.reduce_mean(tf.abs(y_true-y_pred))
mae = tf.keras.metrics.mean_absolute_error(y_true,y_pred)

if mae.ndim > 0:
  mae = tf.reduce_mean(mae)

mae.numpy()
```

RMSE: forecast methods minimizing the RMSE lead to forecasts of the mean.

```
mse = tf.keras.metrics.mean_squared_error(y_true,y_pred)
rmse = tf.sqrt(mse)

if mae.ndim > 0:
  rmse = tf.reduce_mean(rmse)

rmse.numpy()
```

### using a CNN in a time series problem

In [ ]:
# use RNNs and CNNs for time series, sequence, and forecasting models

# to make a CNN:
# before we pass our data to the conv1D layer, we need to reshape it
x = tf.constant(train_windows[0])

model = tf.keras.models.Sequential([
    layers.Lambda(lambda x: tf.expand_dims(x,axis=1)),
    layers.Conv1D(filters=128,kernel_size=5,padding="causal",activation="relu"),
    layers.Dense(HORIZON)
],name="model")

# with ordinary compiling and fitting (see above, under "time series prediction")


### using an RNN (LSTM) in a time series problem


In [ ]:
# to make a RNN (LSTM):
# we'll use the functional API
x = tf.constant(train_windows[0])

inputs = layers.Input(shape=(WINDOW_SIZE))
x = layers.Lambda(lambda x: tf.expand_dims(x,axis=1))(inputs)
x = layers.LSTM(128,activation="relu")(x) # get a massive error with activation="tanh"
# could try adding dense layers to improve performance, 
# like x = layers.Dense(32,activation="relu")(x)
# in that case, layers.LSTM requires an additional return_sequences=True
output = layers.Dense(HORIZON)(x)
model = tf.keras.Model(inputs,outputs,name="model")

# with ordinary compiling and fitting (see above, under "time series prediction")


### time series forecasting

In [ ]:
# identify when to use trailing versus centered windows

# use centered windows when directionality doesn't matter
# in time series, when doing forecasting, you probably need a trailing average

# # i might institute this with something like...
# bitcoin_prices["Moving_Average"] = None

# AVG_KERNEL = 3

# # create a window of specific averaging kernel size
# window_step = np.expand_dims(np.arange(AVG_KERNEL),axis=0)

# # create a 2D array of multiple window steps
# window_indices = window_step + np.expand_dims(np.arange(len(prices)-(WINDOW_SIZE+HORIZON-1)),axis=0).T

# # index on the time series with 2D array
# windowed_array = prices[window_indices]

# # finally, label windowed data
# full_windows, full_labels = windowed_array[:,:-HORIZON], x[:,-HORIZON:]

# # need train data to be early data and test data to be later data
# split_size = int(0.8*len(prices)) # 80% train, 20% test









In [ ]:
# use tensorflow for forecasting

forecast_model = tf.keras.models.Sequential([
    layers.Dense(128,kernel_initializer="he_normal",activation="relu"),
    layers.Dense(128,kernel_initializer="he_normal",activation="relu"),
    layers.Dense(HORIZON)
],name="forecast_model")

forecast_model.compile(loss="mae",
                       optimizer=tf.keras.optimizers.Adam(),
                       metrics=["mae"])

model = forecast_model
data_input = prices
forecast_depth = 100

# take in the model
projection_record = data_input

for i in range(forecast_depth):
  # recalculate all windows and labels
  # create a window of specific window_size
  window_step = np.expand_dims(np.arange(WINDOW_SIZE+HORIZON),axis=0)

  # create a 2D array of multiple window steps
  window_indices = window_step + np.expand_dims(np.arange(len(prices)-(WINDOW_SIZE+HORIZON-1)),axis=0).T

  # index on the time series with 2D array
  windowed_array = prices[window_indices]

  # finally, label windowed data
  all_windows, all_labels = windowed_array[:,:-HORIZON], x[:,-HORIZON:]

  # make combined dataset
  all_dataset_windows = tf.data.Dataset.from_tensor_slices(all_windows)
  all_dataset_labels = tf.data.Dataset.from_tensor_slices(all_labels)
  all_dataset = tf.data.Dataset.zip((all_dataset_windows,all_dataset_labels)) 
  all_dataset = all_dataset.batch(128).prefetch(tf.data.AUTOTUNE)

  # train the model on all of the data
  model.fit(all_dataset,
            epochs=epochs,
            verbose=0)

  # predict next value based on last WINDOW_SIZE price values
  predictor_window = projection_record[-WINDOW_SIZE:]
  next_prediction = model.predict(tf.expand_dims(predictor_window,axis=0))

  # add predicted price value to end of price list
  projection_record = np.append(projection_record,next_prediction)

projection_record


In [ ]:
# identify and compensate for sequence bias

"""
sequence bias is prejudice or favor toward something due to its order within a list
(essentially, where appropriate, you want to randomize the order of your train set...
this is not possible in time series, i would think...)
"""

In [ ]:
# adjust the learning rate dynamically in time series, sequence, 
# and prediction models

"""
OPTIONS:
include ReduceLROnPlateau callback during fit
include LearningRateScheduler callback during fit
"""

### getting time series data with python csv

In [ ]:
# time series with python csv:
import csv
from datetime import datetime

timesteps = []
btc_price = []

with open("/content/BTC_USD_2013-10-01_2021-05-18-CoinDesk.csv", "r") as f:
  csv_reader = csv.reader(f, delimiter=",") # read in the target CSV
  next(csv_reader) # skip first line (this gets rid of the column titles)

  for line in csv_reader:
    timesteps.append(datetime.strptime(line[1], "%Y-%m-%d")) # get the dates as dates (not strings), strptime = string parse time
    btc_price.append(float(line[2])) # get the closing price as float


In [ ]:
timesteps = timesteps
prices = btc_price

### time series train/test sets with python csv

In [ ]:
# need train data to be early data and test data to be later data
split_size = int(0.8*len(prices)) # 80% train, 20% test
X_train, y_train = timesteps[:split_size], prices[:split_size]
X_test, y_test = timesteps[split_size:], prices[split_size:]


## Callback construction

### ModelCheckpoint

save the best weights for the model you're fitting

In [ ]:
import os

save_path = "model_experiments"
# need model_name
tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(save_path,model_name),
                                            verbose=0,
                                            monitor="val_loss",
                                            save_best_only=True)

### ReduceLROnPlateau

when val_loss plateaus, try to improve performance by reducing the learning rate

In [ ]:
tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",
                                     patience=100,
                                     verbose=1)

### LearningRateScheduler

allows us to find the ideal learning rate

In [ ]:
# traverse a set of learning rate values starting from 1e-4, increasing by 10**(epoch/20) every epoch
tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10**(epoch/20))


### EarlyStopping

stops epochs when val_loss is no longer decreasing

In [ ]:
tf.keras.callbacks.EarlyStopping(monitor="val_loss",
                                 patience=200,
                                 restore_best_weights=True)

### TensorBoard

In [ ]:
import datetime

dir_name = "model_logs"
# need experiment_name
log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir)

# To upload TensorBoard dev records:
# !tensorboard dev upload --logdir ./model_logs \
#   --name "First deep model on text data" \
#   --description "Trying a dense model with an embedding layer" \
#   --one_shot # exits the uploader when upload has finished

# If you need to remove previous experiments:
# !tensorboard dev delete --experiment_id EXPERIMENT_ID_TO_DELETE


## If you get stuck, try these to improve performance

* Change learning rate, or use a learning rate-related callback
* Change number of layers in model
* Change number of neurons per layer
* Change the activation functions
* Change the optimization function
* Fit on more data
* Fit for longer
* Normalize the data
* Try data augmentation
* Change batching?
* With time series data, try a trailing moving average?
* Try a Dropout layer?